In [1]:
!pip install --pre deepchem
!pip install rdkit-pypi
import deepchem as dc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 14.7 MB/s eta 0:00:00


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
import deepchem as dc

# Define the splitters to test
splitters = ['random', 'scaffold', 'butina', 'fingerprint']

# Use the roc_auc_score function directly as the metric
metric = dc.metrics.roc_auc_score

# Iterate over each splitter
for splitter in splitters:
    print(f"Testing splitter: {splitter}")

    # Load the Tox21 dataset with the specified splitter
    tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='ECFP', splitter=splitter)
    train_dataset, valid_dataset, test_dataset = datasets

    # Create a MultitaskClassifier model
    model = dc.models.MultitaskClassifier(n_tasks=len(tasks), n_features=1024, layer_sizes=[1000])

    # Train the model
    model.fit(train_dataset, nb_epoch=10)

    # Evaluate the model on the training and test sets
    train_score = model.evaluate(train_dataset, [metric], transformers)
    test_score = model.evaluate(test_dataset, [metric], transformers)

    # Print the results
    print('Training set score:', train_score)
    print('Test set score:', test_score)
    print()

Testing splitter: random


[17:53:56] WARNING: not removing hydrogen atom without neighbors


Training set score: {'metric-1': 0.9546953427340291}
Test set score: {'metric-1': 0.7807381203772606}

Testing splitter: scaffold


[17:54:17] WARNING: not removing hydrogen atom without neighbors
[17:54:25] WARNING: not removing hydrogen atom without neighbors


Training set score: {'metric-1': 0.9580416123911176}
Test set score: {'metric-1': 0.6826106235202589}

Testing splitter: butina


[17:54:39] WARNING: not removing hydrogen atom without neighbors
[17:54:46] WARNING: not removing hydrogen atom without neighbors


Training set score: {'metric-1': 0.958792857050717}
Test set score: {'metric-1': 0.6074208851454063}

Testing splitter: fingerprint


[17:55:10] WARNING: not removing hydrogen atom without neighbors
[17:55:18] WARNING: not removing hydrogen atom without neighbors


Training set score: {'metric-1': 0.9547007546672815}
Test set score: {'metric-1': 0.617588176272305}



In [3]:
tasks, datasets, transformers = dc.molnet.load_hiv(featurizer='ECFP', splitter='scaffold')
train_dataset, valid_dataset, test_dataset = datasets

[18:01:20] WARNING: not removing hydrogen atom without neighbors
[18:01:20] WARNING: not removing hydrogen atom without neighbors
[18:01:46] WARNING: not removing hydrogen atom without neighbors
[18:01:46] WARNING: not removing hydrogen atom without neighbors


In [17]:
params_dict = {
    'n_tasks': [len(tasks)],
    'n_features': [1024],
    'layer_sizes': [[500], [1000], [2000]],
    'dropouts': [[0.5],[0.2]],
    'learning_rate': [0.001, 0.0001]
}

In [9]:
!pip install pyGPGO

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 61.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of pymc3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.5/869.5 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 12.6 MB/s eta 0:00:00
  Created wheel for pyGPGO: filename=pyGPGO-0.5.1-py3-none-any.whl size=19861 sha256=7e30eb8d49141d42cbaec20f6298b705ed29c26dddf900607124c140b7110439
  Stored in directory: /root/.cache/pip/wheels/af/38/ea/ebc23867108cb1f852bb53f5502afb1dce8f55535aea426457
  Created wheel for Theano-PyMC: filename=Theano_PyMC-1.1.2-py

In [18]:
n_tasks = len(tasks)
n_features = 1024
optimizer = dc.hyper.GaussianProcessHyperparamOpt(dc.models.MultitaskClassifier)
metric = dc.metrics.roc_auc_score
best_model, best_hyperparameters, all_results = optimizer.hyperparam_search(params_dict,
                                                                            train_dataset, valid_dataset,
                                                                            metric, transformers)

Evaluation 	 Proposed point 	  Current eval. 	 Best eval.


TypeError: MultitaskClassifier.__init__() missing 2 required positional arguments: 'n_tasks' and 'n_features'